<a href="https://colab.research.google.com/github/wenxuan0923/My-notes/blob/master/collaborative_filtering_TMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recommender System with Collaborate Filtering Method

In this note I will build a recommender system with **Collaborative Filtering (CF)** method using **Surprise** Library in Python. 
Collaborative Filtering method predicts unknown ratings by utilizing the similarities between users: users are recommended items that are being liked by people with similar tastes and interests. 

<a target='_balnk' href='https://surprise.readthedocs.io/en/stable/'>Surprise</a> is a Python library which provides us an easy way to implement and evaluate recommender systems using their built-in prediction algorithms like baseline algorithms, neighborhood methods, matrix factorization-based (SVD, PMF, SVD++, NMF), etc.

The Dataset used in this note is available on Kaggle: <a target='_blank' href='https://www.kaggle.com/rounakbanik/the-movies-dataset'>The Movies Dataset</a>


There are mainly two types of collaborative filtering:

1. **User Based** Collaborative Filtering
> Recommend items based on the users who have similar tastes with the target user. We can measure the **similarity between users** using metrics like pearson correlation or cosine similarity. A potential problem for this methods is: users' preference may change over time.

2. **Item Based** Collaborative Filtering
> Recommend items based on their similarity with the items the target user already rated. Again, the **similarity between items** can be calculated using pearson correlation or cosine similarity.

However, a problem with this method is the sparsity of the user-item matrix (each row is an user and each column an item). High-dimensional datasets are at risk of being very sparse, meaning most training instances are likely to be far away from each other. This also means that a new instance will likely be far away from any training instance, making prediction much less reliable than in lower dimensions. Also, the computation complexity is $O(mn)$ with $m$ users and $n$ items, which will make the training to be very computationally expensive. This is known as the problem of **scalability**.

## Latent Matrix Factorization

One way to handle the scalability and sparsity issue is to leverage a **Latent Factor Model** a.k.a. **Latent Matrix Factorization**. Specifically, it will actually turn the orignal recommendation problem into an optimization problem by predicting the score of each item given by each user. 
The objective function in this case is **Root Mean Square Error (RMSE)**, which is used to measure how good we are in predicting the rating for items given by users. The lower the RMSE, the better the performance.

So what is latent factors? It can be understood as a property or concept that a user or an item have. For example, the genre a movie belongs to. We map each user and each item (see the user-item matrix below) into a lower ($k$) dimensional space, called latent space,  using  **Singular value decomposition** (SVD).

<br>
<img src='https://drive.google.com/uc?id=1tHcKIRV_PByz2uac_-eMMUWOhuq4Anxp'></img>


We’ll first randomly initialize two matrices from a Gaussian Distribution. The first one is the user profiles $P$, a $(m * k)$ matrix, and the second is the movie profiles $Q$, a $k * n$ matrix. The multiplication of these two matrices will result in an $(m * n)$ matrix, and that is the estimated User-Item matrix we want.

Thus, the predicted rating $\hat{r}_{ui}$ of item $i$ by user $u$ can be calculated as:

$$\hat{r}_{ui} = \sum_{j=1}^{k}P_{uj}Q_{ji} = q_i^Tp_u$$

That is the dot product of the original ratings’ user’s row in $P$ and its item’s column in $Q$. To get the best prediction result, we want to minimize:

$$J = \sum_{r_{ui}\in \text{Traing set}}(r_{ui} - \hat{r}_{ui})^2 = (r_{ui}-q_i^Tp_u)^2$$ <br>


We will then optimize the values of $P$ and $Q$ using **Stochastic Gradient Descent**. From the objective function we have:

<br>
$$\frac{\partial J}{\partial p_u} \propto (r_{ui}-\hat{r}_{ui}).q_i$$

$$\frac{\partial J}{\partial q_i} \propto (r_{ui}-\hat{r}_{ui}).p_u$$<br>


Let's denote: $e_{ui} = r_{ui} - \hat{r}_{ui}$, then for each iteration, we will update the values:

<br>
$$p_u := p_u + \gamma(e_{ui} . q_i)$$

$$q_i := q_i + \gamma(e_{ui} . p_u)$$
<br>

where $\gamma$ is the learning rate.

All the expressions above are actually an **unbiased** version of this algorithm. The biased version is simply:

<br>
$$\hat{r}_{ui} = \mu + b_u + b_i + q_i^Tp_u$$<br>

Besides, just like all the ML and DL models, we can add regulariazation terms into our cost function to reduce overfitting. In that case, we minimize the following **regularized squared error**: 

<br>
$$
\begin{align}
J &= \sum_{r_{ui}\in \text{Traing set}}(r_{ui}-\hat{r}_{ui})^2 + \lambda(b_i^2 + b_u^2+||q_i||^2+||p_u||^2) \\
&= \sum_{r_{ui}\in \text{Traing set}} (r_{ui}-\mu - b_u - b_i - q_i^Tp_u)^2 + \lambda(b_i^2 + b_u^2+||q_i||^2+||p_u||^2)
\end{align}
$$ <br>

Where $\lambda$ is the regularization term. The corresponding SGD process will become:

<br>
$$b_u := b_u + \gamma(e_{ui} - \lambda b_u)$$

$$b_i := b_i + \gamma(e_{ui} - \lambda b_i)$$

$$p_u := p_u + \gamma(e_{ui} . q_i - \lambda p_u)$$

$$q_i := q_i + \gamma(e_{ui} . p_u - \lambda q_i)$$
<br>

Once we get the matrixs $P$ and $Q$, we can make prediction of any movie given by any user.

Now Let's see how to implement this method with the help of Surprise library.


## Movie Recommendation Problem

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

I will be using the ratings_small.csv file for model building: it is the subset of 100,000 ratings from 700 users on 9,000 movies.

We'll be using the **Surprise** library to implement and evaluate SVD, and generate prediction result.

In [2]:
df = pd.read_csv('ratings_small.csv')
df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


## Exploratory Data Analysis

### Ratings Distribution

In [150]:
data = df['rating'].value_counts().sort_index(ascending=False)
trace = go.Bar(x = data.index,
               text = ['{:.1f} %'.format(val) for val in (data.values / df.shape[0] * 100)],
               textposition = 'outside',
               marker_color='#aec3d6',
               textfont = dict(color = '#000000'),
               y = data.values,
               )
# Create layout
layout = dict(width=700, height=500,
              plot_bgcolor='#ffffff',
              title = {'text':'Distribution Of {} movie ratings'.format(df.shape[0]),
                       'y':0.9, 'x':0.5},
              xaxis = dict(title = 'Rating', dtick = 0.5),
              yaxis = dict(title = 'Count'))
# Create plot
fig = go.Figure(data=[trace], layout=layout)
fig.update_yaxes(showgrid=True, gridwidth=0.001, gridcolor='#d9dcde')
fig.show()

From this barchart we can see the most common ratings are 4 and 3, they occured 28.7% and 20.1% of the time repectively.

### Distribution of Number of Ratings 

In [54]:
df.groupby('movieId')['rating'].count().sort_values(ascending=False)

movieId
356       341
296       324
318       311
593       304
260       291
         ... 
26467       1
26471       1
26480       1
26485       1
163949      1
Name: rating, Length: 9066, dtype: int64

In [165]:
data = df.groupby('movieId')['rating'].count()
# Create trace
trace = go.Histogram(x = data.values,
                     marker_color='#8bb4d9',
                     name = 'Ratings')
# Create layout
layout = go.Layout(width=900, height=400,
                   title = {'text':'Number of Ratings per Movie',
                            'y':0.9, 'x':0.5},
                   plot_bgcolor='#ffffff',
                   xaxis = dict(title = 'Number of Ratings per Movie'),
                   yaxis = dict(title = 'Count'),
                   bargap = 0.1)

# Create plot
fig = go.Figure(data=[trace], layout=layout)
fig.update_yaxes(showgrid=True, gridwidth=0.001, gridcolor='#d9dcde')
fig.show()

Most of the (3063) movies received only 1 rating, and very few movies have more than 60 ratings, and the most rated movie has 341 ratings.

In [158]:
df.groupby('userId')['rating'].count().sort_values(ascending=False)

userId
547    2391
564    1868
624    1735
15     1700
73     1610
       ... 
444      20
445      20
448      20
498      20
1        20
Name: rating, Length: 671, dtype: int64

In [160]:
data = df.groupby('userId')['rating'].count()
# Create trace
trace = go.Histogram(x = data.values,
                     marker_color='#8bb4d9',
                     name = 'Ratings')
# Create layout
layout = go.Layout(width=900, height=400,
                   title = {'text':'Number of Ratings per User',
                            'y':0.9, 'x':0.5},
                   plot_bgcolor='#ffffff',
                   xaxis = dict(title = 'Number of Ratings per User'),
                   yaxis = dict(title = 'Count'),
                   bargap = 0.2)

# Create plot
fig = go.Figure(data=[trace], layout=layout)
fig.update_yaxes(showgrid=True, gridwidth=0.001, gridcolor='#d9dcde')
fig.show()

Most of the (244) users rate only 1 movie, and very few user rated more than 500 movies, the person who rate the most movies rated 2391 movies.

## Implementation with Surprise
In Surprise we can simply use the built-in `SVD()` method we implment this method. Some important attributes are:

- `n_factors` – The number of factors. Default is 100
- `n_epochs` – The number of iteration of the SGD procedure. Default is 20.
- `biased` (bool) – Whether to use biases. See note above. Default is True.
- `lr_all` – The learning rate for all parameters. Default is 0.005.

We can use the built-in GridSearchCV method to turn these hyperparameters.

Check the <a target='_blank' src='https://surprise.readthedocs.io/en/stable/matrix_factorization.html?highlight=SVD#surprise.prediction_algorithms.matrix_factorization.SVD'>official documentation</a> for more details.

In [237]:
# !pip install surprise
from surprise import Reader, Dataset, SVD, accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection.search import GridSearchCV
from surprise.model_selection.split import train_test_split

reader = Reader()
data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)

### Grid Search Cross Validation

In [238]:
parameters = {'n_factors':[50, 100, 150],  
              'n_epochs':[5, 10, 20], 
              'lr_all':[0.01, 0.005, 0.001]}

In [241]:
gs = GridSearchCV(SVD, parameters, measures=['RMSE'], cv=3)
gs.fit(data)
# best RMSE score
print('Best RMSE Score:', gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print('Best combination of parameters:', gs.best_params['rmse'])

Best RMSE Score: 0.8993561491285323
Best combination of parameters: {'n_factors': 50, 'n_epochs': 20, 'lr_all': 0.005}


Great! We got a mean Root Mean Sqaure Error of 0.8993561491285323 in the cross validation. Let's now prepare a training (80%) and testing set(20%) to further train & test use the algorithm that yields the best rmse.

### Model Training

In [242]:
trainset, testset = train_test_split(data, test_size=0.20)
algo = gs.best_estimator['rmse']
predictions = algo.fit(trainset).test(testset)
accuracy.rmse(predictions)

RMSE: 0.8864


0.8864166690381816

### Make prediction with trained model & evaluation

We have now finished training our model and Root Mean Squared Error is 0.8984. Let now try to make prediction for `usedId=577` and `movieId=2078`. Note the true rating is 5.0. 

In [243]:
df[(df.userId==577) & (df.movieId==2078)]

,userId,movieId,rating,timestamp
86299,577,2078,5.0,1111550822


In [213]:
algo.predict(uid=577, iid=2078) 

Prediction(uid=577, iid=2078, r_ui=None, est=4.727264909481669, details={'was_impossible': False})

The estimated rating is 4.72726 approx, which is very close to the true rating 5! Let's now randomly sample one movie (movieId) which hasn't been rated by user 577 and make prediction:

In [266]:
from random import sample 
sample([id for id in set(df.movieId) if id not in df[df.userId==577].movieId], 1)

[6595]

In [267]:
algo.predict(uid=577, iid=6595) 

Prediction(uid=577, iid=6595, r_ui=None, est=3.8693023664021813, details={'was_impossible': False})

The predicted score is approx 3.8693. We don't know the true value of it, but we can take a look at the histogram of the scores given by other users:


In [271]:
def movie_rating(movidID):
  data = df.loc[df['movieId'] == movidID]['rating']
  trace = go.Histogram(x = data.values,
                      marker_color='#aec3d6',
                      name = 'Ratings')
  # Create layout
  layout = go.Layout(width=700, height=450,
                    plot_bgcolor='#ffffff',
                    title = {'text':'Distribution of Ratings for Movie: {}'.format(movidID),
                              'y':0.9, 'x':0.5},
                    xaxis = dict(title = 'Ratings', dtick = 0.5),
                    yaxis = dict(title = 'Count'),
                    bargap = 0.2)
  # Create plot
  fig = go.Figure(data=[trace], layout=layout)
  fig.update_xaxes(showgrid=True, gridwidth=0.001, gridcolor='#d9dcde')
  fig.update_yaxes(showgrid=True, gridwidth=0.001, gridcolor='#d9dcde')
  fig.show()

In [272]:
movie_rating(movidID = 6595)

From the barchart we can see only most people gave movie 6595 a relatively low rate (< 3). It seems make sense to get a score of 3.8693 though. Now let's evaluate on the whole test set:

In [273]:
def get_Iu(uid):
    """ return the number of items rated by given user
    args: 
      uid: the id of the user
    returns: 
      the number of items rated by the user
    """
    try:
        return len(trainset.ur[trainset.to_inner_uid(uid)])
    except ValueError: # user was not part of the trainset
        return 0
    
def get_Ui(iid):
    """ return number of users that have rated given item
    args:
      iid: the raw id of the item
    returns:
      the number of users that have rated the item.
    """
    try: 
        return len(trainset.ir[trainset.to_inner_iid(iid)])
    except ValueError:
        return 0
    
pred_result = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])
pred_result['Iu'] = pred_result.uid.apply(get_Iu)
pred_result['Ui'] = pred_result.iid.apply(get_Ui)
pred_result['err'] = abs(pred_result.est - pred_result.rui)
best_predictions = pred_result.sort_values(by='err')[:10]
worst_predictions = pred_result.sort_values(by='err')[-10:]

Where

- **uid**: user ID
- **iid**: item ID (Movie Id in our case)
- **rui**: real score given to iid by uid
- **est**: estimated score given to iid by uid
- **Iu**: the number of items rated by the given user
- **Ui**: the number of users that have rated given item
- **err**: error = real score - estimated score (est - rui)


In [274]:
# best prediction: lowest error
best_predictions

,uid,iid,rui,est,details,Iu,Ui,err
3215,287,58559,5.0,5.0,{'was_impossible': False},202,94,0.0
4899,242,912,5.0,5.0,{'was_impossible': False},320,98,0.0
13550,46,26614,5.0,5.0,{'was_impossible': False},31,10,0.0
18747,89,260,5.0,5.0,{'was_impossible': False},52,231,0.0
7898,287,48780,5.0,5.0,{'was_impossible': False},202,40,0.0
19292,46,49530,5.0,5.0,{'was_impossible': False},31,31,0.0
6712,556,318,5.0,5.0,{'was_impossible': False},20,247,0.0
16639,242,1945,5.0,5.0,{'was_impossible': False},320,24,0.0
17806,443,293,5.0,5.0,{'was_impossible': False},32,104,0.0
18590,543,260,5.0,5.0,{'was_impossible': False},30,231,0.0


In [275]:
# worst prediction: highest error
worst_predictions

,uid,iid,rui,est,details,Iu,Ui,err
8552,546,4492,0.5,4.198112,{'was_impossible': False},53,3,3.698112
1035,418,778,0.5,4.206412,{'was_impossible': False},124,104,3.706412
2354,599,71033,0.5,4.208239,{'was_impossible': False},154,3,3.708239
2091,106,46976,0.5,4.219370,{'was_impossible': False},36,34,3.719370
498,282,2959,0.5,4.221999,{'was_impossible': False},99,159,3.721999
16287,83,5902,0.5,4.237427,{'was_impossible': False},124,35,3.737427
16368,646,955,1.0,4.760634,{'was_impossible': False},140,23,3.760634
1017,145,300,1.0,4.862227,{'was_impossible': False},29,72,3.862227
11614,599,4973,0.5,4.564345,{'was_impossible': False},154,100,4.064345
17691,304,527,0.5,4.699840,{'was_impossible': False},91,195,4.199840


Some of the worst predicted samples have very low Ui (like **546**, **599**), meaning very few people have rated this movie, so we don't have encough information to rate these movies. Let's take an example from the worst predictions and dig a little deeper:
User **145** rated item **300** to be 1.0. However, our SVD model predicts this user would rate 4.8622, which leads to an error of 3.8622. Let's take a look at the distribution of the rating from the person who ever rated this movie.

In [276]:
movie_rating(movidID = 300)

Most users gave movie 1722 a relatively high score (> 3), only very few users rated somewhere between 1-2.5. It seems that for each prediction, the users are some kind of outsiders.

**Good References:**

- <a target='_blank' href='https://surprise.readthedocs.io/en/stable/index.html'>Surprise’ documentation</a>

- <a target='_blank' href='https://towardsdatascience.com/building-and-testing-recommender-systems-with-surprise-step-by-step-d4ba702ef80b'>Building and Testing Recommender Systems With Surprise, Step-By-Step</a>